# What is Dash?

### Dash is an open source python framework for building data science and machine learning web applications. It is built on top of Plotly, Flask, and React.
### Provides UI features directly into your python code. 
### https://dash.plotly.com

### pip install dash

## Beginning of an app

In [1]:
from dash import Dash, html

# initialize app
app = Dash()

# define layout
app.layout = [html.Div()]

# run app
if __name__ == '__main__':
    app.run(debug=True)

## App with Data

In [7]:
from dash import Dash, html, dash_table
import pandas as pd

# read csv
df = pd.read_csv('https://raw.githubusercontent.com/roualdes/data/refs/heads/master/penguins.csv')

# initialize app
app = Dash()

# define layout 
app.layout = html.Div([
    dash_table.DataTable(
        data=df.to_dict('records'), # convert panda df to use in dash data table
        page_size=10, # num of rows
        editable=True, # make values editable
        columns=[{'name': col, 'id': col, 'editable': True} for col in df.columns] # define column names and ids
    )
])

# run app
if __name__ == '__main__':
    app.run(debug=True)

## App with Data and Graph

In [9]:
from dash import Dash, html, dash_table, dcc
import pandas as pd
import plotly.express as px

# read csv
df = pd.read_csv('https://raw.githubusercontent.com/roualdes/data/refs/heads/master/penguins.csv')

# initialize app
app = Dash()

# define app layout
app.layout = [
    dcc.Graph(figure=px.violin(df, x='species', y='body_mass_g', color='species'))
]

# run app
if __name__ == '__main__':
    app.run(debug=True)



## App with Data, Graph, and Controls

In [19]:
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import pandas as pd

# read csv
df = pd.read_csv('https://raw.githubusercontent.com/roualdes/data/refs/heads/master/penguins.csv')

# initialize app
app = Dash()

# define app layout
app.layout = [
    html.Div(),
    html.Hr(),
    dcc.RadioItems(options=['body_mass_g', 'bill_depth_mm', 'bill_length_mm'], value='body_mass_g', id='penguin-controls'),
    dcc.Graph(figure={}, id='graph')
]

# add controls to update graph output based on user input
@callback(
    Output(component_id='graph', component_property='figure'),
    Input(component_id='penguin-controls', component_property='value')
)

def update_graph(val_chosen):
    fig = px.box(df, x='species', y=val_chosen) # build graph based off value chosen
    return fig

# run app
if __name__ == '__main__':
    app.run(debug=True)

## App with Data, Graph, and Drop Down

In [21]:
from dash import Dash, html, dcc, callback, Output, Input
import pandas as pd

# read csv
df = pd.read_csv('https://raw.githubusercontent.com/roualdes/data/refs/heads/master/penguins.csv')

# initialize app
app = Dash()

# define app layout
app.layout = html.Div([
    html.Hr(),
    dcc.Dropdown(
        options=[
            {'label': 'Body Mass (g)', 'value': 'body_mass_g'},
            {'label': 'Bill Depth (mm)', 'value': 'bill_depth_mm'},
            {'label': 'Bill Length (mm)', 'value': 'bill_length_mm'},
            {'label': 'Flipper Length (mm)', 'value': 'flipper_length_mm'},
        ],
        value='body_mass_g',
        id='dropdown',
    ),
    dcc.Graph(figure={}, id='graph')
])

# input and output controls to update based off user input
@callback(
    Output(component_id='graph', component_property='figure'),
    Input(component_id='dropdown', component_property='value')
)
def update_graph(val_chosen):
    fig = px.strip(df, x='species', y=val_chosen) # build graph based off species and attribute selected
    return fig

# run app
if __name__ == '__main__':
    app.run(debug=True)

## App with Data, Graph, and Search

In [23]:
from dash import Dash, html, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px

# Read csv
df = pd.read_csv('https://raw.githubusercontent.com/roualdes/data/refs/heads/master/penguins.csv')

# initialized app
app = Dash()

# define the layout 
app.layout = html.Div([
    html.Hr(),
        dcc.Input(
        id='search-species',
        type='text',
    ),
    dcc.Graph(figure={}, id='graph')
])

# input and output controls to change output based on user input
@callback(
    Output(component_id='graph', component_property='figure'),
    Input(component_id='search-species', component_property='value')
)
def update_graph(searched_species):

    if not searched_species: # if nothing is typed view all data 
        filtered_df = df
    else:     # filter data based on species entered
        filtered_df = df[df['species'].str.contains(searched_species, case=False)]

    # build the scatter plot
    fig = px.scatter( 
        filtered_df,
        x='bill_length_mm',
        y='bill_depth_mm',
        color='species',
    )
    return fig


# run app
if __name__ == '__main__':
    app.run(debug=True)

## UFC EVENT/FIGHTER ANALYZER

In [24]:
from dash import Dash, dash_table, dcc, html, Input, Output
import pandas as pd
import plotly.express as px

# read ufc csv
df = pd.read_csv('https://raw.githubusercontent.com/Greco1899/scrape_ufc_stats/refs/heads/main/ufc_fight_stats.csv')

# parse data, fill na values with 0, and convert to int
df['SIG.STR.'] = df['SIG.STR.'].str.split(' of ').str[0].fillna('0').astype(int)
df['STR. LANDED'] = df['TOTAL STR.'].str.split(' of ').str[0].fillna('0').astype(int)
df['STR. THROWN'] = df['TOTAL STR.'].str.split(' of ').str[1].fillna('0').astype(int)
df['TD ATTEMPT'] = df['TD'].str.split(' of ').str[1].fillna('0').astype(int)
df['TD LANDED'] = df['TD'].str.split(' of ').str[0].fillna('0').astype(int)
df['SUB.ATT'] = df['SUB.ATT'].fillna(0).astype(int)

# generate list of fight nights
events = df['EVENT'].unique()

# Initialize Dash app
app = Dash()

app.layout = html.Div([
    html.Div(children='Fighter Statistics', style={'fontSize': 24, 'textAlign': 'center'}),
    html.Hr(),
    
    # dropdown to select ufc event
    dcc.Dropdown(
        options=[{'label': event, 'value': event} for event in events],  # generate list of events for drop down menu
        value=events[0],  # set default event to most recent
        id='ufc-events',
        style={'width': '50%'},
    ),

    # search bar to search for fighters
    dcc.Input(
        id='search-fighter',
        type='text',
        style={'width':'50%', 'margin': '20px 0'},
    ),
    
    # all attributes to select and look at
    dcc.RadioItems(
        options=[{'label': 'Strikes Landed', 'value': 'STR. LANDED'},
                 {'label': 'Strikes Thrown', 'value': 'STR. THROWN'},
                 {'label': 'Significant Strikes', 'value': 'SIG.STR.'},
                 {'label': 'Knockdowns', 'value': 'KD'},
                 {'label': 'Takedowns Landed', 'value': 'TD LANDED'},
                 {'label': 'Takedowns Attempted', 'value': 'TD ATTEMPT'},
                 {'label': 'Submission Attempts', 'value': 'SUB.ATT'}],
        value='STR. LANDED',  # initial value 
        id='radio-items',
        style={'margin': '20px 0'},
    ),
    
    # geneate fighter data table
    dash_table.DataTable(
        id='fighters-table',
        page_size=6,
    ),
    
    # generate graph
    dcc.Graph(figure={}, id='graph')
])

# input and outputs to change data table and graph based on user input
@app.callback(
    [
        Output(component_id='fighters-table', component_property='data'),
        Output(component_id='graph', component_property='figure')
    ],
    [
        Input(component_id='ufc-events', component_property='value'),
        Input(component_id='radio-items', component_property='value'),
        Input(component_id='search-fighter', component_property='value'),
    ]
)
def update_data(selected_event, val_chosen, search_fighter):
    
    # if search bar is used, filter based off fighter entered
    if search_fighter:
        filtered_df = df[df['FIGHTER'].str.contains(search_fighter, case=False, na=False)]
    else:
        filtered_df = df # else use all data
    
    # if drop down menu is used filter based off the event selected
    if selected_event:
        filtered_df = filtered_df[filtered_df['EVENT'] == selected_event]
    
    # update data table based off selected_event or search_fighter
    data_table = filtered_df.to_dict('records')

    # if nothing is entered in either drop down menu or search bar return empty table to avoid error
    if not search_fighter and not selected_event:
        return data_table, {'data': []}
    # if figher is searched display their data for all events associated
    if search_fighter:
        fighter_data = df[df['FIGHTER'].str.contains(search_fighter, case=False, na=False)]
        fig = px.histogram(
            fighter_data,
            x='EVENT',  
            y=val_chosen, # The selected stat on the y-axis
            histfunc = 'sum',
            color='FIGHTER',  # Color by fighter to distinguish different fighters
        )
    # if a fighter is not searched show data for all fighters on that event
    else: 
        fighter_data = filtered_df
        fig = px.histogram(
            fighter_data,
            x='FIGHTER',  
            y=val_chosen,  
            color='EVENT',  
            histfunc='sum',
        )

    return data_table, fig

# run app
if __name__ == '__main__':
    app.run(debug=True)